In [160]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

airbnb_files = ['data/raw_data/Austin_listings.csv', 'data/raw_data/Boston_listings.csv', 'data/raw_data/Asheville_listings.csv']


In [173]:
def drop_airbnb_cols(filename):
    df = pd.read_csv(filename)
    df.drop(['id', 'listing_url', 'scrape_id', 'last_scraped', 'thumbnail_url', 'medium_url', 'picture_url', 'xl_picture_url', 'host_id', 'host_url', 'host_thumbnail_url', 'host_picture_url', 'calendar_last_scraped', 'weekly_price', 'monthly_price', 'neighbourhood_cleansed', 'license', 'jurisdiction_names'], axis=1, inplace=True)
    return df

def get_col_names(files):
    counts = {}
    
    for f in files: 
        for c in drop_airbnb_cols(f).columns: 
            if c in counts: 
                counts[c] += 1
            else: 
                counts[c] = 1

    cols = []
    for c in counts: 
        if counts[c] == 3:
            cols.append(c)
    
    return cols

def segment(df, train, dev, test): 
    count = len(df.index)
    test = test.append(df[-1*int(count*.1):])
    dev = test.append(df[-1*int(count*.2):-1*int(count*.1)])
    train = test.append(df[:int(count*.8)])
    
    return train, dev, test
    
def featurize(df):
    
    df['price'] = df['price'].map(lambda x: x.replace('$', "").replace(',',""))
    df['price'] = df['price'].apply(pd.to_numeric) # turn the price col into a number col
    text_cols = df.select_dtypes(exclude=['float64', 'int64'])
    num_cols = df.select_dtypes(include=['float64', 'int64'])
    
    for col in text_cols: 
        corpus = df[col].fillna(value="").values
        print corpus
        vectorizer = CountVectorizer()
        X = vectorizer.fit_transform(corpus)
        print X

    
    
    # call dictvectorizor on that list --> numpy array of features
    # convert rest of df into numpy array with price as first value
    # append 2 numpy arrays by index
    
    # return big numpy array
    

In [174]:
def create_datasets():
    col_names = get_col_names(airbnb_files)
    
    train = pd.DataFrame(columns = col_names)
    dev = pd.DataFrame(columns = col_names)
    test = pd.DataFrame(columns = col_names)
    
    for f in airbnb_files:   
        df = pd.read_csv(f)
        df = df[col_names]
        featurize(df)
        train, dev, test = segment(df, train, dev, test) # put data from f into train, dev, test
        
    return train, dev, test

In [175]:
train, dev, test = create_datasets()

['2011-03-21' '' '' ..., '2015-05-19' '' '']
  (0, 28)	1
  (0, 2)	1
  (0, 23)	1
  (7, 33)	1
  (7, 9)	1
  (7, 27)	1
  (9, 15)	1
  (9, 27)	1
  (9, 2)	1
  (11, 13)	1
  (11, 26)	1
  (11, 9)	1
  (12, 14)	1
  (12, 7)	1
  (12, 27)	1
  (13, 11)	1
  (13, 4)	1
  (13, 26)	1
  (14, 16)	1
  (14, 24)	1
  (14, 2)	1
  (16, 6)	1
  (16, 5)	1
  (16, 27)	1
  (19, 17)	1
  :	:
  (5822, 2)	1
  (5823, 6)	1
  (5823, 5)	1
  (5823, 27)	1
  (5824, 38)	1
  (5824, 7)	1
  (5824, 27)	1
  (5826, 29)	1
  (5826, 8)	1
  (5826, 26)	1
  (5827, 16)	1
  (5827, 26)	1
  (5827, 2)	1
  (5828, 16)	1
  (5828, 27)	1
  (5828, 2)	1
  (5829, 8)	1
  (5829, 4)	1
  (5829, 27)	1
  (5830, 35)	1
  (5830, 3)	1
  (5830, 27)	1
  (5832, 18)	1
  (5832, 4)	1
  (5832, 27)	1
['f' 't' 't' ..., 'f' 't' 't']


ValueError: empty vocabulary; perhaps the documents only contain stop words